# Vocal tract results analysis
This notebook is used to analyse the results of the simulations of the vocal tract excited by a dirac.

In [13]:
# Imports
%load_ext autoreload
%autoreload 2
#%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
import sympy as sp
import pickle

import os, sys

#Direct input 
plt.rcParams['text.latex.preamble']= r"\usepackage{color}"
#Options
params = {'text.usetex' : True,
          'font.size' : 18,
          'font.family' : 'lmodern',
          'ps.usedistiller' : 'xpdf'
          }
plt.rcParams.update(params) 

p = os.path.abspath('..')
sys.path.insert(1, p)

import rpm_solver as solver
import models
import matplotlib as mpl
%matplotlib qt5

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
# General functions
def mean_state(x):
    """Returns middle values of x for each frame
    (case n_proj = 1)

    Args:
        x (array): array

    Returns:
        array: middle values
    """
    return (x[:-1]+x[1:])/2

def plot_responses(filenames, log = False):
    fig = plt.figure(figsize=(12, 6))
    plt.xlabel('Frequency (Hz)')
    plt.ylabel('Gain (dB)')
    plots = [0 for i in range(len(filenames))]

    nfiles = len(filenames)
    
    norm = mpl.colors.Normalize(vmin=-2, vmax = nfiles-1)
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.PuRd)

    for i, filename in enumerate(filenames):
        with open(filename, 'rb') as f:
            data = pickle.load(f)
        #input_mass_flow = mean_state(data['Inputs'][:, 0, 0])
        input_mass_flow = data['Outputs'][:, 0, 0]
        #output_mass_flow = mean_state(data['Dissipations flows'][:, -1, 0])
        output_mass_flow = data['Dissipations flows'][:, -1, 0]
        sr = data['Parameters']['fs']
        # Frequency vector
        f = np.linspace(0, sr/2, int(np.floor(len(input_mass_flow)/2)+1))
        FFT_in = np.fft.fft(input_mass_flow)[0:int(np.floor(len(input_mass_flow)/2)+1)]
        FFT_out = np.fft.fft(output_mass_flow)[0:int(np.floor(len(input_mass_flow)/2)+1)]

        #Transfer function
        H = 10*np.log10(np.abs(FFT_out)/np.abs(FFT_in))
        N_tracts = data['Parameters']['Tracts']
        plots[i] = plt.plot(f, H, label = r'$N_{tract} = N_tracts$'.replace('N_tracts', f"{N_tracts}"), color = cmap.to_rgba(i))
    plt.gca().legend(loc='upper center', bbox_to_anchor=(0.5, -0.15),
          fancybox=True, shadow=True, ncol=4)
    if log:
        plt.xscale('log')
    plt.tight_layout()
    return fig, plots


## Case Ntract = 10
We look at the differences in the results with a fixed number of tracts but a varying simulation sampling frequency.

In [ ]:
# Filenames of the simulation results
filenames = [f'results/dirac/10_tracts/{sr}_Hz.pkl' for sr in [2000, 5000, 10000, 20000]]

# Transfer functions
fig1, plots1 = plot_responses(filenames, log=False)
plt.xlim(0, 12500)
plt.ylim(-10, 7)
#fig1.suptitle('Transfer functions for 10 tracts with varying sampling frequency')

## Case sr = 20000 Hz, varying number of tracts
Here, we fix the sampling frequency at 20000 Hz and we change the number of tract used for the simulation.

In [ ]:
# Filenames of the simulation results
filenames = [f'results/dirac/40kHz/{N}_tracts_enthalpy_1_idealrad.pkl' for N in [2*(i+1) for i in range(8)]]

# Transfer functionsOutputs
fig2, plots2 = plot_responses(filenames, log=False)
#fig2.suptitle('Transfer functions for N tracts with fixed sampling frequency')
plt.xlim(0, 12500)
plt.ylim(-10, 7)
plt.vlines([1000*i for i in range(20)], -10, 7, linestyles='--')

## Quadrature order

In [ ]:
# Filenames of the simulation results
orders = [1, 2, 4, 8, 16]
filenames = [f'results/dirac/quad_orders/{order}.pkl' for order in orders]

# Transfer functions
fig4, plots4 = plot_responses(filenames, log=False)
for i, plot in enumerate(plots4):
    plot[0].set_label(f'Quadrature order = {orders[i]}')
    plt.legend()

fig4.suptitle('Transfer functions for 17 tracts with changing quadrature order')

## A nice example : 10 tracts with quadrature order 4 and fs = 40kHz

In [ ]:
# Transfer functions
fig5, plots5 = plot_responses(['results/dirac/10_tracts_40k_quad4.pkl'], log=False)

fig5.suptitle('Transfer function for 10 tracts, fs = 40 kHz')
plt.xlim([0, 10000])
plt.ylim([-3, 5])

## Analysis of oscillations due to the constraints

In [ ]:
from cProfile import label

%matplotlib qt5
with open('results/dirac/40kHz/2_tracts_vflow.pkl', 'rb') as f:
    data = pickle.load(f)
    t = data["Time"]
    Pstored = data["Pstored"]
    Pdiss = data["Pdiss"]
    Pext  = data["Pext"]
    Ptot = data["Ptot"]
    #input_mass_flow = mean_state(data['Inputs'][:, 0, 0])
    input_mass_flow = data['Inputs'][:, 0, 0]
    #output_mass_flow = mean_state(data['Dissipations flows'][:, -1, 0])
    output_mass_flow = data['Dissipations flows'][:, -1, 0]
    sr = data['Parameters']['fs']
plt.figure()
#plt.plot(t, Pstored, label=r'$P_{stock}$')
#plt.plot(t, Pdiss, label=r'$P_{diss}$')
#plt.plot(t, Pext, label=r'$P_{ext}$')
#plt.plot(t, Ptot, label=r'$P_{tot}$', c='r')
plt.plot(mean_state(t), mean_state(Pstored), label=r'$P_{stock}$')
plt.plot(mean_state(t), mean_state(Pdiss), label=r'$P_{diss}$')
plt.plot(mean_state(t), mean_state(Pext), label=r'$P_{ext}$')
plt.plot(mean_state(t), mean_state(Ptot), label=r'$P_{tot}$', c='r')
plt.xlabel(r"Temps (s)")
plt.ylabel(r"Puissance (W)")
plt.xlim(0, 10e-3)
plt.legend()

In [63]:
with open('results/dirac/40kHz/2_tracts_vflow.pkl', 'rb') as f:
    data = pickle.load(f)
    t = data["Time"]
    Pstored = data["Pstored"]
    Pdiss = data["Pdiss"]
    Pext  = data["Pext"]
    Ptot = data["Ptot"]
    E1 = data["Projected gradH"][:, 1]
    E2 = data["Projected gradH"][:, 6]
    #input_mass_flow = mean_state(data['Inputs'][:, 0, 0])
    input_mass_flow = data['Outputs'][:, 0, 0]
    qL1 = data['States'][:,6]
    qR2 = data['States'][:,7]
    flows = data['States flows']
    #output_mass_flow = mean_state(data['Dissipations flows'][:, -1, 0])
    output_mass_flow = data['Dissipations flows'][:, -1, 0]
    sr = data['Parameters']['fs']
plt.figure()
#plt.plot(t, Pstored, label=r'$P_{stored}$')
#plt.plot(t, Pdiss, label=r'$P_{diss}$')
#plt.plot(t, Pext, label=r'$P_{ext}$')
#plt.plot(t, Ptot, label=r'$P_{tot}$', color='r')
plt.plot(mean_state(t), mean_state(qL1), label=r'$\nu_R^1$')
plt.plot(mean_state(t), mean_state(qR2), label=r'$\nu_L^2$')
#plt.plot(t, qL1, label=r'$\nu_R^1$')
#plt.plot(t, qR2, label=r'$\nu_L^2$')
#plt.plot(t, qR, label=r'$\nu_R^1$')
plt.xlabel(r"Temps (s)")
plt.ylabel(r"Débit surfacique ($m^2.s^{-1}$)")
plt.xlim(0, 10e-3)
#plt.ylim(-1e-12, 4e-12)
#plt.title('p=4')
plt.legend()
plt.tight_layout()